In [1]:
path = "C:\\Users\\vaish\\Downloads\\shape_predictor_68_face_landmarks.dat"
import cv2
import dlib

# Load the pre-trained face detector
face_detector = dlib.get_frontal_face_detector()

# Load the facial landmark predictor
landmark_predictor = dlib.shape_predictor(path)

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_detector(gray)

    for face in faces:
        # Detect facial landmarks for each face
        landmarks = landmark_predictor(gray, face)

        # Extract specific landmarks for eyes and nose
        left_eye_center = ((landmarks.part(36).x + landmarks.part(39).x) // 2, (landmarks.part(36).y + landmarks.part(39).y) // 2)
        right_eye_center = ((landmarks.part(42).x + landmarks.part(45).x) // 2, (landmarks.part(42).y + landmarks.part(45).y) // 2)
        nose_tip = (landmarks.part(30).x, landmarks.part(30).y)

        # Draw circles at the detected landmarks
        cv2.circle(frame, left_eye_center, 2, (0, 255, 0), -1)
        cv2.circle(frame, right_eye_center, 2, (0, 255, 0), -1)
        cv2.circle(frame, nose_tip, 2, (0, 255, 0), -1)

        # Calculate the direction the user is looking
        direction = "Unknown"
        if nose_tip[1] < min(left_eye_center[1], right_eye_center[1]) - 10:  # Eyes are looking up
            direction = "Up"
        elif nose_tip[1] > max(left_eye_center[1], right_eye_center[1]) + 10:  # Eyes are looking down
            direction = "Down"
        elif nose_tip[0] < min(left_eye_center[0], right_eye_center[0]) - 10:  # Eyes are looking left
            direction = "Left"
        elif nose_tip[0] > max(left_eye_center[0], right_eye_center[0]) + 10:  # Eyes are looking right
            direction = "Right"

        # Display the direction text on the frame
        cv2.putText(frame, f"Looking: {direction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


'wget' is not recognized as an internal or external command,
operable program or batch file.
'bunzip2' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import cv2
import numpy as np

# For the face and eye detection load the Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
threshold_angle = 20  # degrees

def detect_eyes_and_face(gray_frame, colored_frame):
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_color = colored_frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        cv2.rectangle(colored_frame, (x,y), (x+w, y+h), (255, 0, 0), 4)
        face_center = (x+w//2, y+h//2)
        
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            center = (x+ex+ew // 2, y+ey+eh // 2)
            cv2.circle(colored_frame, center, 2, (0, 0, 255), -1)
            eye_center = (ex+ew // 2, ey+eh // 2)
            
            # getting the direction of gaze
            direction = detect_corner_gaze(face_center, eye_center, x, y)
            print(direction)
        
        if eye_gaze(colored_frame, x, y, w, h):
            # Notify the user.
            print("User is looking away from the camera.")
    
    return colored_frame

def eye_gaze(frame, x, y, w, h):
    eye_region = frame[y:y+h, x:x+w]
    eye_region_gray = cv2.cvtColor(eye_region, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(eye_region_gray, scaleFactor=1.1, minNeighbors=5)
    # If no eyes are detected, then the user is looking away from the camera.
    if len(eyes) == 0:
        return True
    # Otherwise, the user is looking at the camera.
    return False

def gaze_direction(face_center, eye_center):
    angle = np.arctan2(eye_center[1] - face_center[1], eye_center[0] - face_center[0])
    angle = angle * 180 / np.pi
    return angle

def detect_corner_gaze(face_center, eye_center, x, y):
    # Calculate gaze angle using gaze_direction function
    gaze_angle = gaze_direction(face_center, eye_center)
    # Define screen corners (simplified for demonstration)
    top_left_angle = np.arctan2(y, -x) * 180 / np.pi  # Example angle for top-left corner
    bottom_right_angle = np.arctan2(-y, x) * 180 / np.pi  # Example angle for bottom-right corner
    # Define thresholds for each corner (adjust as needed)
    threshold_angle = 10  # Example threshold angle
    # Check if gaze angle falls within threshold range for each corner
    if abs(gaze_angle - top_left_angle) <= threshold_angle:
        return "Looking at top-left corner"
    elif abs(gaze_angle - bottom_right_angle) <= threshold_angle:
        return "Looking at bottom-right corner"
    else:
        return "Not looking at corners"

def is_looking_at_screen(face_center, eye_center):
    angle = gaze_direction(face_center, eye_center)
    return abs(angle) <= threshold_angle


# open webcam
webcam_capture = cv2.VideoCapture(0)
print("""Press "q" to Exit.""")
while(True):
    ret, video_frame = webcam_capture.read()
    if(ret is False):
        break
    gray_frame = cv2.cvtColor(video_frame, cv2.COLOR_BGR2GRAY)
    result = detect_eyes_and_face(gray_frame, video_frame)
    cv2.imshow("Face and Eye", result)
    
    if(cv2.waitKey(1) & 0XFF==ord('q')):
        break

webcam_capture.release()
cv2.destroyAllWindows()

Press "q" to Exit.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
User is looking away from the camera.
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at corners
Not looking at cor